In [1]:
#Writing the import statements
import pandas as pd   
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
import re
import json
import enchant
d = enchant.Dict("en_US")
from collections import Counter
from nltk.stem import PorterStemmer, WordNetLemmatizer
import numpy as np
#This is used for stemming of words
stemmer = PorterStemmer()


import nltk
nltk.download("stopwords")

In [2]:
#Defining function for converting a raw review to a string of words 

def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    #
    # 1. Remove HTML
    print("Removing HTML")
    review_text = BeautifulSoup(raw_review,"lxml").get_text() 
    #
    # 2. Remove non-letters
    print("Removing Non-Letters")
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    print("Converting the words to Lower Case")
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    print("Converting the stopwords to a set")
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    print("Removing the stopwords")
    meaningful_words = [w for w in words if not w in stops]   
    #
    #6.Remove Non-English words
    print("Removing the non-English Words")
    meaningful_words_2 = [w for w in meaningful_words if d.check(w)]
    print("Preprocessing of raw data is completed")
    #7.stemming
    #meaningful_words_3=[stemmer.stem(w) for w in  meaningful_words_2]
    # 8. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words_2))

In [19]:
# Read the data from disk and split into lines
# we use .strip() to remove the final (empty) line

def readData():
    print("Gathering Raw Data")
    with open("review_partial.json",encoding="utf8") as f:
        reviews = f.read().strip().split("\n")
    reviews = [json.loads(review) for review in reviews]
    print("Gathered the Raw Data")
    return(reviews)
    
def getTextsAndStars(reviews):
    print("Getting the Raw Text and the rating corresponding to the texts")
    texts = [review['text'] for review in reviews]
    stars = [review['stars'] for review in reviews]
    #Writing the texts and stars into a dataframe for later use
    textsAndStars = pd.DataFrame(columns=['texts','stars'])
    textsAndStars['texts'] = texts
    textsAndStars['stars'] = stars
    print("Obtained the raw text and ratings")
    return(textsAndStars)


In [20]:
reviews = readData()

Gathering Raw Data
Gathered the Raw Data


In [38]:
#reviews

In [22]:
textsAndStars = getTextsAndStars(reviews)

Getting the Raw Text and the rating corresponding to the texts
Obtained the raw text and ratings


In [13]:
#Calling the readData function to obtain the texts and stars from the json file
#textsAndStars = readData()
texts = textsAndStars['texts']
stars = textsAndStars['stars']

In [7]:
#Getting all reviews for the user with most reviews
texts = [review['text'] for review in reviews if review['user_id'] == "CxDOIDnH8gp9KXzpBHJYXw"]
#Getting all ratings for the user with most reviews
stars = [review['stars'] for review in reviews if review['user_id'] == "CxDOIDnH8gp9KXzpBHJYXw"]

In [26]:
#Getting all reviews for certain businesses
texts = [review['text'] for review in reviews if review['business_id'] == "uYHaNptLzDLoV_JZ_MuzUA"]
#Getting all ratings for the user with most reviews
stars = [review['stars'] for review in reviews if review['business_id'] == "uYHaNptLzDLoV_JZ_MuzUA"]

In [28]:
len(texts)

16

In [30]:
#texts

['My girlfriend and I stayed here for 3 nights and loved it. The location of this hotel and very decent price makes this an amazing deal. When you walk out the front door Scott Monument and Princes street are right in front of you, Edinburgh Castle and the Royal Mile is a 2 minute walk via a close right around the corner, and there are so many hidden gems nearby including Calton Hill and the newly opened Arches that made this location incredible.\n\nThe hotel itself was also very nice with a reasonably priced bar, very considerate staff, and small but comfortable rooms with excellent bathrooms and showers. Only two minor complaints are no telephones in room for room service (not a huge deal for us) and no AC in the room, but they have huge windows which can be fully opened. The staff were incredible though, letting us borrow umbrellas for the rain, giving us maps and directions, and also when we had lost our only UK adapter for charging our phones gave us a very fancy one for free.\n\n

In [31]:
#removing stopwords, special characters
def clean_Reviews():
    print("Preparing to clean reviews")
    clean_reviews = []
    for i in range(len(texts)):
        # If the index is evenly divisible by 1000, print a message showing how many reviews have been cleaned
        if( (i+1)%5000 == 0 ):
            print ("Review %d of %d\n" % ( i+1, num_reviews ))                                                                    
        clean_reviews.append( review_to_words( texts[i]))
        print("Cleaned Reviews obtained")
    return(clean_reviews)

In [34]:
clean_reviews = []
clean_reviews = clean_Reviews()

Preparing to clean reviews
Removing HTML
Removing Non-Letters
Converting the words to Lower Case
Converting the stopwords to a set
Removing the stopwords
Removing the non-English Words
Preprocessing of raw data is completed
Cleaned Reviews obtained
Removing HTML
Removing Non-Letters
Converting the words to Lower Case
Converting the stopwords to a set
Removing the stopwords
Removing the non-English Words
Preprocessing of raw data is completed
Cleaned Reviews obtained
Removing HTML
Removing Non-Letters
Converting the words to Lower Case
Converting the stopwords to a set
Removing the stopwords
Removing the non-English Words
Preprocessing of raw data is completed
Cleaned Reviews obtained
Removing HTML
Removing Non-Letters
Converting the words to Lower Case
Converting the stopwords to a set
Removing the stopwords
Removing the non-English Words
Preprocessing of raw data is completed
Cleaned Reviews obtained
Removing HTML
Removing Non-Letters
Converting the words to Lower Case
Converting the 

In [37]:
#Checking the pre-processed text
clean_reviews[0]

'girlfriend stayed nights loved location hotel decent price makes amazing deal walk front door monument princes street right front castle royal mile minute walk via close right around corner many hidden gems nearby including hill newly opened arches made location incredible hotel also nice reasonably priced bar considerate staff small comfortable rooms excellent bathrooms showers two minor complaints telephones room room service huge deal us ac room huge windows fully opened staff incredible though letting us borrow umbrellas rain giving us maps directions also lost adapter charging phones gave us fancy one free would highly recommend hotel friends return definitely staying without hesitation'

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abhishek/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

def create_Vectorizer():
    # Initialize the "CountVectorizer" object, which is scikit-learn's
    # bag of words tool.
    print ("Creating the bag of words...\n")

    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, 
                                 max_features = 20000) 


    # fit_transform() does two functions: First, it fits the model
    # and learns the vocabulary; second, it transforms our training data
    # into feature vectors. The input to fit_transform should be a list of 
    # strings.
    data_features = vectorizer.fit_transform(clean_reviews)

    data_features = data_features.toarray()
    print("Created the bag of words")

    # Take a look at the words in the vocabulary
    vocab = vectorizer.get_feature_names()

    # # Sum up the counts of each vocabulary word
    dist = np.sum(data_features, axis=0)

    #The below code obtains the low frequency words
    threshold_count=2
    low_freq_words=[]
    
    # #for tag, count in zip(vocab, dist):
    low_freq_words=[tag  for tag, count in zip(vocab, dist) if(count<threshold_count)]
    
    return(data_features)

In [12]:
data_features = create_Vectorizer()

Creating the bag of words...

Created the bag of words


In [13]:
#Getting User ID for users

def getUsersAndCounts():
    print("Obtaining a collection of users and number of reviews in order to obtain top users")
    userIds = [review['user_id'] for review in reviews]
    from collections import Counter
    userCounts = Counter(userIds)
    #userCounts.most_common()

In [14]:
getUsersAndCounts()

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tfidf_vectorizer(): 
    print("Preparing to vectorize the text using tf-idf")
    # This vectorizer breaks text into single words and bi-grams and then calculates the TF-IDF representation
    vectorizer = TfidfVectorizer(ngram_range=(1,2))
    # the 'fit' builds up the vocabulary from all the reviews while the 'transform' step turns each indivdual text into
    # a matrix of numbers.
    vectors = vectorizer.fit_transform(clean_reviews)
    print("Completed tf-idf vectorization of text data")
    return(vectors)

In [16]:
vectors = tfidf_vectorizer()

In [17]:
#Cross Validation import
from sklearn.model_selection import cross_val_score

In [18]:
def gaussian_Naive_Bayes():    
    from sklearn.naive_bayes import GaussianNB
    clf = GaussianNB()
    print("Running Gaussian Naive Bayes")
    clf.fit(vectors.toarray(), stars)
    scores = cross_val_score(clf, vectors.toarray(), stars, cv=5)
    print("Getting Scores")
    print(scores)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [19]:
gaussian_Naive_Bayes()

Running Gaussian Naive Bayes
Getting Scores
[ 0.55797101  0.53768116  0.54571843  0.56768559  0.55247813]
Accuracy: 0.55 (+/- 0.02)


In [20]:
def linear_SVM():
    from sklearn.svm import LinearSVC
    classifier = LinearSVC()
    print("Running Linear SVM Classifier")
    classifier.fit(vectors.toarray(), stars)
    scores = cross_val_score(classifier, vectors.toarray(), stars, cv=5)
    print("Getting Scores")
    print(scores)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [21]:
linear_SVM()

Getting Scores
[ 0.61449275  0.61884058  0.61248186  0.61426492  0.62244898]
Accuracy: 0.62 (+/- 0.01)


In [ ]:
def support_Vector_Machine():
    from sklearn.svm import SVC
    classifierSVC = SVC(C=0.5,gamma=0.5)
    print("Running Support Vector Machine Classifier")
    classifierSVC.fit(vectors.toarray(), stars)
    scores = cross_val_score(classifierSVC, vectors.toarray(), stars, cv=5)
    print("Getting Scores")
    print(scores)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
support_Vector_Machine()

In [22]:
def random_Forest_Classifier():
    from sklearn.ensemble import RandomForestClassifier
    classifierRF = RandomForestClassifier()
    print("Running Random Forest Classifier")
    classifierRF.fit(vectors.toarray(), stars)
    scores = cross_val_score(classifierRF, vectors.toarray(), stars, cv=5)
    print("Getting Scores")
    print(scores)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [23]:
random_Forest_Classifier()

Getting Scores
[ 0.54637681  0.53913043  0.55152395  0.55167394  0.55393586]
Accuracy: 0.55 (+/- 0.01)


In [39]:
def tune_linear_svm():
    from sklearn.grid_search import GridSearchCV
    from sklearn.svm import LinearSVC
    print("Obtaining best hyper-parameters for Linear SVM Classifier")
    grid={"C": [0.05,0.5,1,1.5,2,5,10], "loss": ["hinge", "squared_hinge"], "class_weight": [None,"balanced"]}

    train_X = vectors.toarray()
    train_y = stars

    clf = GridSearchCV(LinearSVC(), grid, cv=3).fit(train_X, train_y)

    print(clf.best_params_)
    print(str(clf.best_score_))
    #print("")

    return clf

In [38]:
tune_linear_svm()

{'C': 2, 'class_weight': 'balanced', 'loss': 'squared_hinge'}
0.6202789076118536



GridSearchCV(cv=3, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.05, 0.5, 1, 1.5, 2, 5, 10], 'loss': ['hinge', 'squared_hinge'], 'class_weight': [None, 'balanced']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [40]:
def logistic_Regression():
    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression() 
    model.fit(vectors.toarray(), stars)
    scores = cross_val_score(model, vectors.toarray(), stars, cv=5)
    print("Getting Scores")
    print(scores)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [41]:
logistic_Regression()

Getting Scores
[ 0.56811594  0.56376812  0.57329463  0.56914119  0.57725948]
Accuracy: 0.57 (+/- 0.01)


In [ ]:
reviews = readData()
textsAndStars = getTextsAndStars(reviews)